In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchvision import datasets, transforms
from datetime import datetime
import os
import matplotlib.pyplot as plt
from skimage.util.shape import view_as_windows
device = torch.device("cuda:0" if (torch.cuda.is_available() ) else "cpu")
# device="cpu"
print(device)

cuda:0


In [2]:
# %cd /content/drive/MyDrive/Dresden\ Dataset

In [3]:
pwd

'/run/media/dhanush/Stuff/study/mislnet'

In [4]:
class MislNetModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.ccnn_wts=nn.Parameter(torch.randn(size=[3,1,5,5]), requires_grad=True).to(device)
        self.conv2=nn.Conv2d(1,96,7, stride =2, padding=4)
        self.conv3=nn.Conv2d(96,64,5, stride=1, padding=2)
        self.conv4=nn.Conv2d(64,64,5, stride=1, padding=2)
        self.conv5=nn.Conv2d(64,128,1, stride =1)
        self.fc1=nn.Linear(6272,200) #the first parameter changes with image size
        self.fc2=nn.Linear(200,200)
        self.fc3=nn.Linear(200,27)  #6 is the number of out classes
        self.maxpool=nn.MaxPool2d(3,2)
        self.avgpool=nn.AvgPool2d(3,2)
    
    def remove_content(self):
        for i in range(3):
            self.ccnn_wts.data[i,0,2,2]=0
            self.ccnn_wts.data[i] /= torch.sum(self.ccnn_wts.data[i])
            self.ccnn_wts.data[i,0,2,2]=-1

    def forward(self, x):
        # x=x.to(device)
        self.remove_content()
        F.conv2d(x,self.ccnn_wts)
        x=self.conv2(x)
        x=self.maxpool(x)
        # x=nn.BatchNorm2d(96)(x)
        x=torch.tanh(x)

        x=self.conv3(x)
        x=self.maxpool(x)
        # x=nn.BatchNorm2d(64)(x)
        x=torch.tanh(x)

        x=self.conv4(x)
        x=self.maxpool(x)
        # x=nn.BatchNorm2d(64)(x)
        x=torch.tanh(x)

        x=self.conv5(x)
        x=self.avgpool(x)
        # x=nn.BatchNorm2d(128)(x)
        x=torch.tanh(x)

        x=torch.flatten(x,1)
        # print(x.shape)
        x=self.fc1(x)
        x=torch.tanh(x)
        x=self.fc2(x)
        x=torch.tanh(x)
        x=self.fc3(x)
        x=F.softmax(x,1)
        return x

model=MislNetModel().to(device)
print(model)

MislNetModel(
  (conv2): Conv2d(1, 96, kernel_size=(7, 7), stride=(2, 2), padding=(4, 4))
  (conv3): Conv2d(96, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv4): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv5): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Linear(in_features=6272, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=27, bias=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avgpool): AvgPool2d(kernel_size=3, stride=2, padding=0)
)


In [5]:
# temp3, temp7=torch.utils.data.random_split(range(10),[3,7])
# tempLoader3=torch.utils.data.DataLoader(tempData)
# tempLoader7=torch.utils.data.DataLoader(tempData)
# for data in temp7:
#     print(data)


In [6]:
16960*0.8

13568.0

In [7]:
transform=transforms.Compose([
                              transforms.Resize((256,256)),
                              transforms.Grayscale(),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5), (0.5))
])
dataset=datasets.ImageFolder('./Dresden Dataset/',transform=transform)
# dataloader=torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
num_train=len(dataset)*0.8
num_train=int(num_train)
train_dataset, test_dataset=torch.utils.data.random_split(dataset, [num_train, len(dataset)-num_train])
train_dataloader=torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader=torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)


In [66]:
patch_size=256
for each_image in dataset:
    patches=view_as_windows(fullSize_inputs, (patch_size,patch_size),patch_size)
    print(dataset[1000][1])
    break
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


KeyboardInterrupt: 

In [8]:
images,label=next(iter(train_dataloader))


In [9]:
print(model(images.to(device)).shape)

torch.Size([64, 27])


/home/dhanush/Desktop/btp/pytorch_venv/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [10]:
import torch.optim as optim
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(), momentum=0.95, weight_decay=0.0005, lr=1e-3)

In [13]:
# chkptname=datetime.now()
# chkptname=chkptname.strftime("%d-%m_%H%M")
# chkptname="./checkpoints/{}".format(chkptname)
# print(chkptname)
# os.mkdir(chkptname)
for epoch in range(1):
    running_loss=0
    for i,data in enumerate(train_dataloader,0):
        fullSize_inputs,labels=data
        print(fullSize_inputs.shape)
        break
        patch_size=256
        for single_img in fullSize_inputs:
            patch_inputs=view_as_windows(fullSize_inputs, (patch_size,patch_size),patch_size)
        
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs=model(inputs)
#         print(outputs.shape, labels.shape, min(labels), max(labels))
#         loss=criterion(outputs,labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         if i % 30 == 29:    # print every 30 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 2000))
#             running_loss = 0.0
    print(f"Loss for epoch {epoch} -> {loss.item()}")
    torch.save(model.state_dict(), chkptname+f"/epoch_{epoch+1}.pt")
    # since we're not training, we don't need to calculate the gradients for our outputs


torch.Size([64, 1, 256, 256])


NameError: name 'loss' is not defined

In [13]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # calculate outputs by running images through the network
        outputs = model(inputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 8 %


In [14]:
#play beep sound when done
from playsound import playsound
while 1:
    playsound("./alarm.mp3")

playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.


KeyboardInterrupt: 